# Using Iceberg and Query Engines

This notebook demonstrates how you can use Iceberg and Spark / Trino to query your data.

We will:
- Configure Spark with Iceberg runtime libaries
- Explore how iceberg works
- Explore data we just generated
- Come up with a plan to run a Spark job with Spark Operator


## Setting up Spark and Iceberg

In [1]:
from pyspark.sql import SparkSession
import os

# These packages are necessary for running this section on EKS. We have made these jars available within the container image, therefore the line below is commented out.
# packages = "org.apache.iceberg:iceberg-spark:1.10.0,org.apache.iceberg:iceberg-spark-runtime-4.0_2.13:1.10.0,org.apache.iceberg:iceberg-spark-extensions-4.0_2.13:1.10.0,com.amazonaws:aws-java-sdk-bundle:1.12.791,software.amazon.awssdk:bundle:2.34.0,org.apache.hadoop:hadoop-aws:3.4.1"
spark = SparkSession.builder \
    .appName("IcebergInspector") \
    .config("spark.sql.catalog.workshop", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.workshop.type", "glue") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .config("spark.sql.defaultCatalog", "workshop") \
    .config("spark.sql.catalog.workshop.warehouse", "s3a://data-on-eks-spark-logs-20251001184655839600000005/iceberg-warehouse/") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem").getOrCreate()
    # .config("spark.sql.catalog.workshop.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \


spark.sparkContext.setLogLevel("DEBUG")

## Explore Iceberg Tables

In [2]:
# We read database information from the catalog
spark.sql("show databases").show()
# Then use the database for all queries going forward.
spark.sql("use data_on_eks")
# Show all tables available with in the database
spark.sql("show tables").show()

+-----------+
|  namespace|
+-----------+
|data_on_eks|
+-----------+

+-----------+--------------------+-----------+
|  namespace|           tableName|isTemporary|
+-----------+--------------------+-----------+
|data_on_eks|     cafe_orders_raw|      false|
|data_on_eks|cafe_orders_raw_6...|      false|
|data_on_eks|cat_interactions_raw|      false|
|data_on_eks|cat_interactions_...|      false|
|data_on_eks|   cat_locations_raw|      false|
|data_on_eks|cat_locations_raw...|      false|
|data_on_eks|    cat_wellness_raw|      false|
|data_on_eks|cat_wellness_raw_...|      false|
|data_on_eks|visitor_checkins_raw|      false|
|data_on_eks|visitor_checkins_...|      false|
+-----------+--------------------+-----------+



# Peeking Under the Hood: Exploring Metadata Tables
One of Iceberg's most powerful features is its transparency. Unlike other table formats where the underlying structure is hidden, Iceberg allows us to directly query its metadata to understand exactly what's going on.
We'll use special sub-tables like .files, .history, and .snapshots to inspect the table's physical layout and history.

Let's start by looking at the actual data files on disk. Every time we write data, Iceberg creates one or more files (in this case, Parquet files). The .files metadata table gives us a complete list of every data file that makes up the current snapshot of the table.
This demonstrates how small, frequent writes can lead to multiple small files.



In [4]:
spark.sql("""
SELECT
    file_path, file_format, file_size_in_bytes, record_count
FROM
    spark_catalog.data_on_eks.cat_locations_raw.files
ORDER BY file_path DESC
LIMIT 10
""").show()

+--------------------+-----------+------------------+------------+
|           file_path|file_format|file_size_in_bytes|record_count|
+--------------------+-----------+------------------+------------+
|s3a://data-on-eks...|    PARQUET|             98651|       23820|
|s3a://data-on-eks...|    PARQUET|             99303|       24007|
|s3a://data-on-eks...|    PARQUET|             98269|       23767|
|s3a://data-on-eks...|    PARQUET|             99176|       23930|
|s3a://data-on-eks...|    PARQUET|             98700|       23827|
|s3a://data-on-eks...|    PARQUET|             40429|        6132|
|s3a://data-on-eks...|    PARQUET|             89337|       21136|
+--------------------+-----------+------------------+------------+



## The History Table
The .history table is like a commit log for your data. Every change made to the table—like our INSERT statements—is recorded as an entry. This provides a clear, chronological audit trail of how the table has evolved. The query below will show the timeline of when each snapshot became the current version of the table. C. Show the Snapshot Details (.snapshots) While .history shows the timeline, the .snapshots table gives us the rich details for each snapshot. A snapshot is an immutable view of the table's complete state at a specific point in time. The query below shows what operation created each snapshot (append, overwrite, etc.) and a summary of the changes, like how many records and files were added. This table is the key that enables powerful features like time travel.

The `is_current_ancestor` column is a boolean flag in the history table that answers a simple question: "Is this historical snapshot part of the direct timeline that leads to the table's current state?"

  * If the value is **`true`**, the snapshot is a direct ancestor.
  * If the value is **`false`**, the snapshot belongs to an abandoned branch of history, most often created after a table rollback.

The diagrams below illustrate how this works.

-----

### Scenario 1: Linear History

Initially, your table has three commits (snapshots). Each is a direct ancestor of the current version (`S3`), so `is_current_ancestor` is **true** for all of them.

```
(Main Timeline)
+-------------+      +-------------+      +-------------+
| Snapshot S1 |----->| Snapshot S2 |----->| Snapshot S3 |
| ancestor: T |      | ancestor: T |      | ancestor: T |
+-------------+      +-------------+      +-------------+
                                                    ^
                                                    |
                                                 (current)
```


### Scenario 2: After Rolling Back to S2

Now, you roll the table back to `S2`. The main timeline is now shorter, and `S2` is the new current version.

Snapshot `S3` still exists in the table's history, but it's now on an **abandoned branch**. Its `is_current_ancestor` flag flips to **false**.

```
(Main Timeline)                               (Abandoned Branch)
+-------------+      +-------------+                 +-------------+
| Snapshot S1 |----->| Snapshot S2 |                 | Snapshot S3 |
| ancestor: T |      | ancestor: T |                 | ancestor: F |
+-------------+      +-------------+                 +-------------+
                           ^
                           |
                        (current)

In [16]:
spark.sql("""
SELECT
    *
FROM
    workshop.data_on_eks.cat_locations_raw.history
""").show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2025-10-02 18:16:...| 686702856221647031|               NULL|               true|
|2025-10-02 18:18:...|6250352781994368215| 686702856221647031|               true|
|2025-10-02 18:20:...|7910074619865715724|6250352781994368215|               true|
|2025-10-02 18:22:...| 705454000691502411|7910074619865715724|               true|
|2025-10-02 18:24:...|4643221966159991104| 705454000691502411|               true|
|2025-10-02 18:26:...|2900310610060766013|4643221966159991104|               true|
|2025-10-02 18:28:...| 245501116594841622|2900310610060766013|               true|
|2025-10-02 18:30:...| 656001455602482726| 245501116594841622|               true|
|2025-10-02 18:32:...|2576547903821103422| 656001455602482726|               true|
|202

#### Now that we understand how to view the metadata, let's use it to demonstrate some of Iceberg's most famous features

## Schema Evolution (The "No-Panic" Schema Change)

This is one of Iceberg's most valuable features. It allows you to change a table's schema without rewriting all the existing data.

  * **How to demonstrate:**
    1.  **Show the current schema:**
        ```sql
        DESCRIBE TABLE workshop.`data-on-eks`.daily_cat_summary
        ```
    2.  **Add a new column:** Run an `ALTER TABLE` command. This is a metadata-only operation and will complete instantly.
        ```sql
        ALTER TABLE workshop.`data-on-eks`.daily_cat_summary ADD COLUMN notes STRING
        ```
    3.  **Verify the new schema:**
        ```sql
        DESCRIBE TABLE workshop.`data-on-eks`.daily_cat_summary
        ```
    4.  **Query the data:** Show that the table is still perfectly readable. Old rows will simply have a `null` value for the new `notes` column.
        ```sql
        SELECT day, cat_id, name, notes FROM workshop.`data-on-eks`.daily_cat_summary LIMIT 10
        ```

In [21]:
spark.sql("""
SELECT
    *
FROM
    workshop.data_on_eks.cat_wellness_raw
WHERE
    event_date is not NULL
""").show()

+--------------------+--------------------+--------------+----------+----------------------+----------+
|          event_time|              cat_id|activity_level|heart_rate|hours_since_last_drink|event_date|
+--------------------+--------------------+--------------+----------+----------------------+----------+
|2025-10-02T18:53:...|e288cb79-0721-49b...|          3.16|       102|                   3.6|2025-10-02|
|2025-10-02T18:53:...|bee21146-e74b-4b3...|          8.54|       102|                  2.81|2025-10-02|
|2025-10-02T18:53:...|95421121-7507-478...|          1.22|       111|                  3.34|2025-10-02|
|2025-10-02T18:53:...|960795a9-01f6-4d5...|          7.92|        98|                  2.89|2025-10-02|
|2025-10-02T18:53:...|147ed417-2cda-496...|          1.27|        88|                  2.97|2025-10-02|
|2025-10-02T18:53:...|78d188ea-bc24-4ec...|          9.37|        87|                   2.2|2025-10-02|
|2025-10-02T18:53:...|9438bd05-cc1f-4ad...|          3.76|      

In [4]:
spark.sql("""
DESCRIBE TABLE EXTENDED workshop.data_on_eks.cat_wellness_raw
""").show(100, False)

+----------------------------+----------------------------------------------------------------------------------------------------------------------+-------+
|col_name                    |data_type                                                                                                             |comment|
+----------------------------+----------------------------------------------------------------------------------------------------------------------+-------+
|event_time                  |string                                                                                                                |NULL   |
|cat_id                      |string                                                                                                                |NULL   |
|activity_level              |double                                                                                                                |NULL   |
|heart_rate                  |int                   